In [1]:
import sys
import time
#!{sys.executable} -m pip install inquirer 
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from itertools import product 
import pandas as pd
import os
from selenium.webdriver.common.action_chains import ActionChains
from datetime import datetime
import numpy as np
import itertools
import pathlib
import cv
import random
import re
import matplotlib.pyplot as plt

%matplotlib inline


## Notes

1 - AGE: Age Groups are different in v2 and v3, I made an affort to pair similar groups with the understanding that they are not identical

2 - RACE: Hispanic or Latinos Patient attribute is in a separate attribute in v2 (Patient Ethnicity) and is included  in Patient Race in v3, therefore I created a new column cRace to include Hispanics in attribute discribing Age
In WONDER 'Asian'+ 'Native Hawaiian or Other Pacific Islander' = 'Asian or Pacific Islander' . Need to combine to be able to merge

Duplication in v3 observed when broken by race
Expected 385,231 and got 387,326


3 - GENDER: For both v2 and v3 all values different from Female or Male are set to Unknown in column cGender

4- CENSUS REGION: Island Areas in v2 seems to be renamed to Territories in  v3. I created column cRegion in v2 that replaces value Island Areas with Territories. v3 also Not reported values that might need to be dropped

5- CENSUS DIVISION: Territories is missing from v3. We might have to drop those values

6- MEDICATION: More Naxolone versions observed in v3 than v4. Medication can be dropped


In [2]:
###Census Region correction 
def combine(v2,v3):
    v2.columns=[c.split('.')[-1] for c in v2.columns]
    v3.columns=[c.split('.')[-1] for c in v3.columns]
    v2['Month']= pd.to_datetime(v2['Year - Quarter -  Month - Day'])
    v3['Month']=pd.to_datetime(v3[['Year - Quarter - Month - Day','Month of Year']].apply(lambda x: str(x[0])+'-'+str(x[1]), axis=1))
    if v2['Count of Events'].dtypes=='object':
        v2['Count of Events']=v2['Count of Events'].apply(lambda x: int(x.replace(',','')))
    if v3['Count of Events'].dtypes=='object':
        v3['Count of Events']=v3['Count of Events'].apply(lambda x: int(x.replace(',','')))    
    v2['Region']= v2.apply(lambda x: 'Territories' if x['Census Region']=='Island Areas' 
             else x['Census Region'], axis=1)
    v2_region=v2['Region'].unique()
    v3['Region'] = v3['US Census Regions']
    v2_region.sort()
    v3_region=v3['Region'].unique()
    v3_region.sort()
    v2_month_region = v2.groupby(['Month','Region'])['Count of Events'].sum().reset_index(name='Count of Events')
    v3_month_region = v3.groupby(['Month','Region'])['Count of Events'].sum().reset_index(name='Count of Events')
    v2_month = v2.groupby(['Month'])['Count of Events'].sum().reset_index(name='Count of Events')
    v3_month = v3.groupby(['Month'])['Count of Events'].sum().reset_index(name='Count of Events')

    nemsis_monthly_reg=pd.concat([v2_month_region,v3_month_region],ignore_index=True)
    nemsis_monthly=pd.concat([v2_month,v3_month],ignore_index=True)
    return nemsis_monthly_reg, nemsis_monthly


In [4]:
v2_data = 'newdata/v2_monthly_by_region_Naxolone.csv'
v3_data = 'newdata/v3_monthly_by_region_Naxolone.csv'
v3=pd.read_csv(v3_data)
v2=pd.read_csv(v2_data)
nemsis_monthly_reg_medication, nemsis_monthly_medication = combine(v2,v3)

v2_data = 'newdata/v2_monthly_by_region_977_90.csv'
v3_data = 'newdata/v3_monthly_by_region_F11T40.csv'
v3=pd.read_csv(v3_data)
v2=pd.read_csv(v2_data)
nemsis_monthly_reg_icd, nemsis_monthly_icd = combine(v2,v3)

nemsis_monthly_reg_medication.to_csv("nemsis_v2v3_monthly_reg_medication.csv", index=False)
nemsis_monthly_medication.to_csv("nemsis_v2v3_monthly_medication.csv", index=False)
nemsis_monthly_reg_icd.to_csv("nemsis_v2v3_monthly_reg_icd.csv", index=False)
nemsis_monthly_icd.to_csv("nemsis_v2v3_monthly_icd.csv", index=False)
print(nemsis_monthly_reg_medication['Count of Events'].sum(), nemsis_monthly_medication['Count of Events'].sum())
print(nemsis_monthly_reg_icd['Count of Events'].sum(), nemsis_monthly_icd['Count of Events'].sum())

In [5]:
v2_data ='extracted_data/v2_race_ethnicity_Naxolone_Narcan.csv'
v3_data ='extracted_data/v3_race_Naxolone_Narcan.csv'
v2=pd.read_csv(v2_data)
v3=pd.read_csv(v3_data)
v2.fillna(0)
v3.fillna(0)
if v2['Count of Events'].dtypes=='object':
    v2['Count of Events'] = v2['Count of Events'].apply(lambda x: int(x.replace(',','')))
if v3['Count of Events'].dtypes=='object':
    v3['Count of Events'] = v3['Count of Events'].apply(lambda x: int(x.replace(',','')))
total_v2 = v2['Count of Events'].sum()
total_v3 = v3['Count of Events'].sum()
print('Total Events with Naloxone: ', total_v2+total_v3)    
print('Total v2: {0} \nTotal v3: {1} '.format(total_v2,total_v3))   
# ###Race correction 
v2['Race']= v2.apply(lambda x: 'Hispanic or Latino' if x['Patient Ethnicity']=='Hispanic or Latino' 
         else x['Patient Race'], axis=1)
v2.loc[v2['Race'].isin(['Not Applicable','Not Available','Not Known','Not Recorded'
                              ,'Not Reporting','Other Race']),'Race']='Unknown'
v3['Race']= v3["Race"].apply(lambda x: 'Unknown' if x in ['Not Applicable','Not Available','Not Known'
                                 ,'Not Recorded','Not Reporting','Other Race'] else x)

events_before_agg=v2['Count of Events'].sum()
v2_month_race=v2.groupby(['Month','Race']).apply(lambda x: x['Count of Events'].sum()).reset_index(name='Count of Events')
events_after_agg=v2_month_race['Count of Events'].sum()
assert events_before_agg==events_after_agg

events_before_agg=v3['Count of Events'].sum()
v3_month_race=v3.groupby(['Month','Race']).apply(lambda x: x['Count of Events'].sum()).reset_index(name='Count of Events')
events_after_agg=v3_month_race['Count of Events'].sum()
assert events_before_agg==events_after_agg

v2_race=v2.Race.unique()
v3_race=v3.Race.unique()
v3_race.sort(),v2_race.sort()
assert list(v2_race)==list(v3_race)
v3.groupby('Month')['Count of Events'].sum()
nemsis_month_race=pd.concat([v2_month_race,v3_month_race])

nemsis_month_race=nemsis_month_race.pivot(index='Month', columns='Race' , values='Count of Events')
nemsis_month_race.fillna(0,inplace=True)
nemsis_month_race=nemsis_month_race.astype('int64')
nemsis_month_race.to_csv("nemsis_v2v3_month_race.csv")


Total Events with Naloxone:  882980
Total v2: 495667 
Total v3: 387313 
